# Streaming demo
This notebook demonstrates pre-feature processing connection to Azure SQL Database, manipulation and calculation on gathered MARG sensors data. Prerequisites to this demo is setting environment variables on your computer, which can be found on the team's channel. 

The steps shown below are:
1. Importing modules and functions crucial for creating **jerk**, **magnitude** and **lag** columns.
2. Cleaning Database from old contents, while leaving the necessary accelerometer data from conducted dfs.
3. Downloading accelerometer data and creating DataFrame.
4. Creating new columns.
5. Processing downloaded DataFrame.
6. Sending new data to corresponding database table.
7. Showing results.

For this demo, instead of using variables dependent on the length of the data in the table, we used hard-coded values in some places.

> I recommend running one block at the time while looking at the current table contents using Data Studio. End result is shown in the last cell.

In [22]:
# import modules, functions 
import pyodbc
import pandas as pd 
import numpy as np
import os

from Feats_columns import *

# get data needed for db connection
driver = os.environ['DBDriver']
server = os.environ['DBServer']
database = os.environ['DBDb']
username = os.environ['DBUser']
password = os.environ['DBPass']

In [23]:
# initial db cleaning 
with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cleaningQuery = "ALTER TABLE [dbo].[Accelerometer] DROP COLUMN jerkZ, jerkY, jerkX, jerkMag, mag, lagX0, \
                         lagX1, lagX2, lagX3, lagY0, lagY1,lagY2, lagY3, lagZ0, lagZ1, lagZ2, lagZ3, ID"
        cursor.execute(cleaningQuery)


In [24]:
# download data
with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT * FROM [dbo].[Accelerometer]")
        data = cursor.fetchall()

df = pd.DataFrame.from_records(data, columns = ['time', 'seconds_elapsed', 'z', 'y', 'x'])

In [25]:
# create columns for new data
with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        idQuery = "ALTER TABLE [dbo].[Accelerometer] ADD ID INT IDENTITY(1,1)"
        cursor.execute(idQuery)
        
        createCols = "ALTER TABLE [dbo].[Accelerometer] ADD jerkZ varchar (MAX),  jerkY varchar (MAX),  jerkX varchar (MAX),  jerkMag varchar (MAX),  \
              mag varchar (MAX), lagX0 varchar (MAX),  lagX1 varchar (MAX),  lagX2 varchar (MAX), lagX3 varchar (MAX), \
              lagY0 varchar (MAX),  lagY1 varchar (MAX),  lagY2 varchar (MAX), lagY3 varchar (MAX), \
              lagZ0 varchar (MAX),  lagZ1 varchar (MAX),  lagZ2 varchar (MAX), lagZ3 varchar (MAX)"
        cursor.execute(createCols)

In [26]:
# create new features
jerkM, jerkX, jerkY, jerkZ = jerk(df)
mag = magnitude(df)
laggerX, laggerY, laggerZ = lag(3, df)

In [27]:
# stream data to database
with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        for i in range(len(jerkZ)):
            updateQuery = f"UPDATE [dbo].[Accelerometer] SET jerkZ = {jerkZ[i]}, jerkY = {jerkY[i]}, jerkX = {jerkX[i]}, jerkMag = {jerkM[i]}, mag = {mag[i]}, lagX0 = '{laggerX['lag 0'].iloc[i]}', lagX1 = '{laggerX['lag 1'].iloc[i]}', \
                         lagX2 = '{laggerX['lag 2'].iloc[i]}', lagX3 = '{laggerX['lag 3'].iloc[i]}', lagY0 = '{laggerY['lag 0'].iloc[i]}', lagY1 = '{laggerY['lag 1'].iloc[i]}', lagY2 = '{laggerY['lag 2'].iloc[i]}', lagY3 = '{laggerY['lag 3'].iloc[i]}',  \
                         lagZ0 = '{laggerZ['lag 0'].iloc[i]}', lagZ1 = '{laggerZ['lag 1'].iloc[i]}', lagZ2 = '{laggerZ['lag 2'].iloc[i]}', lagZ3 = '{laggerZ['lag 3'].iloc[i]}' WHERE ID = {i+1};"
            cursor.execute(updateQuery)

        missingUpdateQuery = f"UPDATE [dbo].[Accelerometer] SET mag = {mag[-1]}, lagX0 = {laggerX['lag 0'].iloc[-1]}, lagX1 = {laggerX['lag 1'].iloc[-1]}, lagX2 = {laggerX['lag 2'].iloc[-1]},  \
                         lagX3 = {laggerX['lag 3'].iloc[-1]}, lagY0 = {laggerY['lag 0'].iloc[-1]}, lagY1 = {laggerY['lag 1'].iloc[-1]}, lagY2 = {laggerY['lag 2'].iloc[-1]}, lagY3 = {laggerY['lag 3'].iloc[-1]},  \
                         lagZ0 = {laggerZ['lag 0'].iloc[-1]}, lagZ1 = {laggerZ['lag 1'].iloc[-1]}, lagZ2 = {laggerZ['lag 2'].iloc[-1]}, lagZ3 = {laggerZ['lag 3'].iloc[-1]} WHERE ID = {i+2};"
        cursor.execute(missingUpdateQuery)



In [28]:
# show results
with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        dispQuery = "SELECT TOP (100) [time],[seconds_elapsed],[acl-z],[acl-y],[acl-x],[ID],[jerkZ],[jerkY],[jerkX],[jerkMag], \
                    [mag],[lagX0],[lagX1],[lagX2],[lagX3],[lagY0],[lagY1],[lagY2],[lagY3],[lagZ0],[lagZ1],[lagZ2],[lagZ3] FROM [dbo].[Accelerometer]"
        cursor.execute(dispQuery)
        for row in cursor.fetchall():
            print(row)


(1667085933660507600, 0.077507568359375, 0.1348760879576206, 0.117631523168087, -0.03745294782072305, 1, '0.000595244043557587', '0.00031028279418178836', '7.60736e-005', '0.0009816004846393304', '0.182842712860146', '-0.03745294782072305', 'nan', 'nan', 'nan', '0.117631523168087', 'nan', 'nan', 'nan', '0.1348760879576206', 'nan', 'nan', 'nan')
(1667085933670554600, 0.0875546875, 0.10037263759225606, 0.14254267706274987, -0.025118147708848116, 2, '6.22775e-005', '6.46465e-005', '0.0013442517663945482', '0.0014711757407351795', '0.17613631796357412', '-0.025118147708848116', '-0.03745294782072305', 'nan', 'nan', '0.14254267706274987', '0.117631523168087', 'nan', 'nan', '0.10037263759225606', '0.1348760879576206', 'nan', 'nan')
(1667085933680601600, 0.0976015625, 0.08921221502199768, 0.13117197274267672, -0.0769689291184768, 3, '1.80323e-005', '1.59645e-005', '0.0057543038659132574', '0.00578830063596414', '0.176321075858735', '-0.0769689291184768', '-0.025118147708848116', '-0.037452947